# Yolo 결과로 Arduino 구동하기

## Arduino

아두이노는 다양한 프로젝트를 만들고 제어할 수 있는 오픈 소스 전자 플랫폼이다. 마이크로컨트롤러 보드와 그를 프로그래밍하기 위한 개발 환경으로 구성된다. 마이크로컨트롤러는 아두이노 보드의 두뇌로서 명령을 실행하고 외부 세계와 상호작용하는 역할을 담당한다.

아두이노의 주요 장점 중 하나는 그 간단함과 사용 편의성으로, 초보자와 전문가 모두에게 접근 가능다는 것이다. 아두이노 프로그래밍 언어는 C/C++의 단순화된 버전인 Wiring을 기반으로 하여, 전자 부품을 제어하기 위한 코드를 배우고 작성하는 것이 아주 쉽다.

아두이노를 사용하면 센서, 모터, 라이트 등의 전자 기기를 연결하여 대화형 프로젝트를 만들 수 있다. 코드를 작성하고 아두이노 보드에 업로드함으로써 이러한 기기들을 제어하고 다양한 입력에 대응하거나 특정 작업을 수행하도록 할 수 있다. 로봇을 만들거나 가정 자동화, 웨어러블 기술, 예술 설치물 등을 개발하고자 한다면, 아두이노는 아이디어를 현실로 구현하는 매우 유용한 플랫폼이다.

아두이노 보드는 다양한 크기와 형태로 제공되며, 다양한 프로젝트 요구 사항에 맞추어 선택할 수 있다. 배터리로 구동하거나 USB를 통해 컴퓨터에 연결할 수도 있다. 또한 아두이노 보드는 다양한 쉴드와 모듈과 호환되어 있어 기능을 확장하고 다른 기기와 쉽게 연결할 수 있다.

아두이노의 오픈 소스 특성으로 인해 다양한 사용자 커뮤니티가 만들어졌으며, 이들은 프로젝트, 지식 및 코드를 공유한다. 다양한 튜토리얼, 예제 및 라이브러리를 온라인에서 찾아볼 수 있어 초보자에게도 학습과 문제 해결이 쉽다. 아두이노는 다양성과 가격 대비 성능이 우수하여 교육 목적, 프로토타이핑 및 거의 모든 취미 프로젝트에 사용되고 있다.

<img src="images_larduino/capture00.png" style="width:800px"><br>

## Arduino IDE 설치

- https://www.arduino.cc 에 접속 "SOFTWARE" 탭 클릭

<img src="images_larduino/capture01.png" style="width:800px"><br>

- 설치하고 arduino ide를 구동합니다. 그리고 보드와 포트를 선택합니다.

<img src="images_larduino/capture02.png" style="width:800px"><br>

- servo 모터를 구동하는 예제 프로그램을 열어서 연결한 서보모터가 잘 작동하는지 확인합니다.

<img src="images_larduino/capture04.png" style="width:600px"><br>

<img src="images_larduino/capture03.png" style="width:800px"><br>

## Serial port 명령을 통해서 서보모터를 회전시켜 봅시다.

<img src="images_larduino/capture05.png" style="width:700px"><br>

## 이제 Yolo로 predict한 결과를 해석하고 이걸로 아두이노를 구동해봅시다.

- python에서 serial port를 구동하려면 pyserial 라이브러리를 설치해야 합니다.
    > pip install pyserial

<img src="images_larduino/capture06.png" style="width:800px"><br>

- 그리고 기존의 yolo detection 코드를 약간 수정해 줍니다.

In [ ]:
import serial


from ultralytics import YOLO
import cv2
from ultralytics.yolo.utils.plotting import Annotator
import serial       # 여기 이 줄을 추가!

this_serial = serial.Serial('/dev/ttyUSB0', 9600)   # 이 줄도 추가

# Load a model
# model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
model = YOLO("runs/detect/train/weights/best.pt")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model.predict(source=img, save=False, show=False, conf=0.3)
        for r in results:
            boxes = r.boxes
            annotator = Annotator(frame)
            for box in boxes:
                b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
                c = box.cls
                cnf = box.conf.item()
                b_caption = model.names[int(c)] + f"_({cnf:.2f})"
                annotator.box_label(b, b_caption)

            # 이 if 블록 추가
            if len(boxes) > 0:      
                b = boxes[0]
                c = b.cls
                angle = int(c)*90
                print(angle)
                this_serial.write(f"{angle}\n".encode('utf-8'))
            # 여기까지

        frame = annotator.result() 
        cv2.imshow("yolov8", frame)

        key = cv2.waitKey(1)
        if key > 0:
            break

cap.release()
cv2.destroyAllWindows()

this_serial.close()